In [110]:
# Import required libraries
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")


Libraries imported successfully!


In [111]:
# Database connection configuration
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '3306')
DB_NAME = os.getenv('DB_NAME', 'your_database_name')
DB_USER = os.getenv('DB_USER', 'your_username')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'your_password')

# Create connection string
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

print(f"Connecting to database: {DB_NAME} on {DB_HOST}:{DB_PORT}")
print(f"User: {DB_USER}")

# Test connection
try:
    engine = create_engine(connection_string, echo=False)
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1 as test"))
        print("✅ Database connection successful!")
        print(f"Connection test result: {result.fetchone()[0]}")
except SQLAlchemyError as e:
    print(f"❌ Database connection failed: {e}")
    print("Please check your database credentials in the .env file")


Connecting to database: api_dev on 10.110.32.104:3306
User: bkdjabar
✅ Database connection successful!
Connection test result: 1


In [112]:
def load_data_from_sql(query, engine):
    """
    Load data from MySQL database into a pandas DataFrame.
    
    Parameters:
    query (str): SQL query to execute
    engine: SQLAlchemy engine object
    
    Returns:
    pandas.DataFrame: Data from the query
    """
    try:
        df = pd.read_sql(query, engine)
        print(f"✅ Data loaded successfully! Shape: {df.shape}")
        return df
    except SQLAlchemyError as e:
        print(f"❌ Error loading data: {e}")
        return None

def get_table_info(table_name, engine):
    """
    Get basic information about a table.
    
    Parameters:
    table_name (str): Name of the table
    engine: SQLAlchemy engine object
    """
    try:
        # Get table structure
        structure_query = f"DESCRIBE {table_name}"
        structure = pd.read_sql(structure_query, engine)
        
        # Get row count
        count_query = f"SELECT COUNT(*) as row_count FROM {table_name}"
        count_result = pd.read_sql(count_query, engine)
        
        print(f"📊 Table: {table_name}")
        print(f"Rows: {count_result['row_count'].iloc[0]}")
        print(f"Columns: {len(structure)}")
        print("\nColumn Information:")
        print(structure)
        
        return structure
    except SQLAlchemyError as e:
        print(f"❌ Error getting table info: {e}")
        return None

def list_tables(engine):
    """
    List all tables in the database.
    
    Parameters:
    engine: SQLAlchemy engine object
    """
    try:
        query = "SHOW TABLES"
        tables = pd.read_sql(query, engine)
        print("📋 Available tables:")
        for table in tables.iloc[:, 0]:
            print(f"  - {table}")
        return tables
    except SQLAlchemyError as e:
        print(f"❌ Error listing tables: {e}")
        return None

print("Data loading functions defined successfully!")



Data loading functions defined successfully!


In [113]:
# Load excel data into dataframe

excel_file_path = 'Kepsek.xlsx'
df_kepsek = pd.read_excel(excel_file_path)

df_kepsek.head()

,peg_id,peg_nama,tugas_tambahan_jenis,tugas_tambahan_jabatan,riw_jabatan_tmt,riw_jabatan_no No SK Jabatan,riw_jabatan_tgl Tanggal SK Jabatan,riw_jabatan_pejabat Pejabat penandatangan SK,nm_gol_akhir
0,196709191991031005,"Drs. SAPTO WIBOWO, M.M.",definitif,KEPALA SEKOLAH SMAN 1 JONGGOL KABUPATEN BOGOR,2025-10-29,821.2/Kep.689-BKD/2025,2025-10-28,GUBERNUR JAWA BARAT,IV/a
1,196801011998021003,Drs. MARYANA,definitif,KEPALA SEKOLAH SMAN 1 CIGUDEG KABUPATEN BOGOR,2025-10-29,821.2/Kep.689-BKD/2025,2025-10-28,GUBERNUR JAWA BARAT,IV/a
2,196801041993021002,"ENDANG MISBAH KURNIAWAN, S.Pd.",definitif,KEPALA SEKOLAH SMAN 1 CISARUA KABUPATEN BOGOR,2025-10-29,821.2/Kep.689-BKD/2025,2025-10-28,GUBERNUR JAWA BARAT,IV/b
3,196807071990012001,"RD. NIA ELISA YULIANTI, S.Pd., M.Pd.",definitif,KEPALA SEKOLAH SMAN 1 CIGOMBONG KABUPATEN BOGOR,2025-10-29,821.2/Kep.689-BKD/2025,2025-10-28,GUBERNUR JAWA BARAT,IV/c
4,196905191994122001,"MELWINDA FITRI, S.Pd., M.Pd.",definitif,KEPALA SEKOLAH SMAN 3 CIBINONG KABUPATEN BOGOR,2025-10-29,821.2/Kep.689-BKD/2025,2025-10-28,GUBERNUR JAWA BARAT,IV/b


In [114]:
# Extract NIP, nama, tugas_tambahan
df_kepsek_clean = df_kepsek[['peg_id', 'peg_nama', 'tugas_tambahan_jabatan']]

df_kepsek_clean.head()

,peg_id,peg_nama,tugas_tambahan_jabatan
0,196709191991031005,"Drs. SAPTO WIBOWO, M.M.",KEPALA SEKOLAH SMAN 1 JONGGOL KABUPATEN BOGOR
1,196801011998021003,Drs. MARYANA,KEPALA SEKOLAH SMAN 1 CIGUDEG KABUPATEN BOGOR
2,196801041993021002,"ENDANG MISBAH KURNIAWAN, S.Pd.",KEPALA SEKOLAH SMAN 1 CISARUA KABUPATEN BOGOR
3,196807071990012001,"RD. NIA ELISA YULIANTI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 1 CIGOMBONG KABUPATEN BOGOR
4,196905191994122001,"MELWINDA FITRI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 3 CIBINONG KABUPATEN BOGOR


In [115]:
# Extract sekolah
# Remove 'KEPALA SEKOLAH' from tugas_tambahan_jabatan
df_kepsek_lok = df_kepsek_clean.copy()
# Remove 'KEPALA SEKOLAH' from the beginning and strip spaces to get the school name
df_kepsek_lok['sekolah'] = df_kepsek_lok['tugas_tambahan_jabatan'].str.replace(r'^KEPALA SEKOLAH\s*', '', regex=True).str.strip()
df_kepsek_lok.head()

,peg_id,peg_nama,tugas_tambahan_jabatan,sekolah
0,196709191991031005,"Drs. SAPTO WIBOWO, M.M.",KEPALA SEKOLAH SMAN 1 JONGGOL KABUPATEN BOGOR,SMAN 1 JONGGOL KABUPATEN BOGOR
1,196801011998021003,Drs. MARYANA,KEPALA SEKOLAH SMAN 1 CIGUDEG KABUPATEN BOGOR,SMAN 1 CIGUDEG KABUPATEN BOGOR
2,196801041993021002,"ENDANG MISBAH KURNIAWAN, S.Pd.",KEPALA SEKOLAH SMAN 1 CISARUA KABUPATEN BOGOR,SMAN 1 CISARUA KABUPATEN BOGOR
3,196807071990012001,"RD. NIA ELISA YULIANTI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 1 CIGOMBONG KABUPATEN BOGOR,SMAN 1 CIGOMBONG KABUPATEN BOGOR
4,196905191994122001,"MELWINDA FITRI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 3 CIBINONG KABUPATEN BOGOR,SMAN 3 CIBINONG KABUPATEN BOGOR


In [116]:
# Database connection configuration
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '3306')
DB_NAME = os.getenv('DB_NAME', 'your_database_name')
DB_USER = os.getenv('DB_USER', 'your_username')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'your_password')

# Create connection string
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

print(f"Connecting to database: {DB_NAME} on {DB_HOST}:{DB_PORT}")
print(f"User: {DB_USER}")

# Test connection
try:
    engine = create_engine(connection_string, echo=False)
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1 as test"))
        print("✅ Database connection successful!")
        print(f"Connection test result: {result.fetchone()[0]}")
except SQLAlchemyError as e:
    print(f"❌ Database connection failed: {e}")
    print("Please check your database credentials in the .env file")


Connecting to database: api_dev on 10.110.32.104:3306
User: bkdjabar
✅ Database connection successful!
Connection test result: 1


In [117]:
def load_data_from_sql(query, engine):
    """
    Load data from MySQL database into a pandas DataFrame.
    
    Parameters:
    query (str): SQL query to execute
    engine: SQLAlchemy engine object
    
    Returns:
    pandas.DataFrame: Data from the query
    """
    try:
        df = pd.read_sql(query, engine)
        print(f"✅ Data loaded successfully! Shape: {df.shape}")
        return df
    except SQLAlchemyError as e:
        print(f"❌ Error loading data: {e}")
        return None

def get_table_info(table_name, engine):
    """
    Get basic information about a table.
    
    Parameters:
    table_name (str): Name of the table
    engine: SQLAlchemy engine object
    """
    try:
        # Get table structure
        structure_query = f"DESCRIBE {table_name}"
        structure = pd.read_sql(structure_query, engine)
        
        # Get row count
        count_query = f"SELECT COUNT(*) as row_count FROM {table_name}"
        count_result = pd.read_sql(count_query, engine)
        
        print(f"📊 Table: {table_name}")
        print(f"Rows: {count_result['row_count'].iloc[0]}")
        print(f"Columns: {len(structure)}")
        print("\nColumn Information:")
        print(structure)
        
        return structure
    except SQLAlchemyError as e:
        print(f"❌ Error getting table info: {e}")
        return None

def list_tables(engine):
    """
    List all tables in the database.
    
    Parameters:
    engine: SQLAlchemy engine object
    """
    try:
        query = "SHOW TABLES"
        tables = pd.read_sql(query, engine)
        print("📋 Available tables:")
        for table in tables.iloc[:, 0]:
            print(f"  - {table}")
        return tables
    except SQLAlchemyError as e:
        print(f"❌ Error listing tables: {e}")
        return None

print("Data loading functions defined successfully!")



Data loading functions defined successfully!


In [118]:
query = "SELECT * FROM m_gps \
    WHERE id_opd = '1003'"

db_gps = load_data_from_sql(query, engine)
db_gps.head()

✅ Data loaded successfully! Shape: (1653, 10)


,gps_id,id_opd,nama,lat,lon,id_unitkerja,is_pusat,cek_data,del_or_keep,deleted_at
0,36,1003,SMAN 1 CIBINGBIN,-7.0556,108.75004,100330120200,0,digunakan,keep,NaT
1,255,1003,SMAN 9 GARUT,-7.060808,108.090775,100331113000,0,digunakan,keep,NaT
2,260,1003,SLBN LURAGUNG,-7.0153516,108.6194041,100330126000,0,digunakan,keep,NaT
3,261,1003,SMAN 1 ARJAWINANGUN,-6.639608,108.405377,100330110100,0,digunakan,keep,NaT
4,262,1003,SMAN 1 ASTANAJAPURA,-6.7884868,108.6179578,100330110200,0,digunakan,keep,NaT


In [119]:
query = "SELECT * FROM m_unitkerja \
    WHERE id_opd = '1003'"

db_unitkerja = load_data_from_sql(query, engine)
db_unitkerja.head()

✅ Data loaded successfully! Shape: (13607, 12)


,id,id_unitkerja,id_unitkerja_atasan,unitkerja_nama,nama_jabatan,keselon,unkerjagrade,id_opd,unor,nip_pejabat,is_uptd,kode_simpeg
0,10180,1003,None,DINAS PENDIDIKAN,None,None,A,1003,None,None,0,None
1,10181,100301000000,1003,SEKRETARIAT,None,None,A,1003,None,None,0,None
2,10182,100301010000,100301000000,SUBBAGIAN PERENCANAAN DAN PELAPORAN,None,None,A,1003,None,None,0,None
3,10183,100301020000,100301000000,SUBBAGIAN KEUANGAN DAN ASET,None,None,A,1003,None,None,0,None
4,10184,100301030000,100301000000,"SUBBAGIAN KEPEGAWAIAN, UMUM, DAN KEHUMASAN",None,None,A,1003,None,None,0,None


In [120]:
# Get ID unit kerja from sekolah
df_combine = pd.merge(df_kepsek_lok, db_unitkerja, left_on='sekolah', right_on='unitkerja_nama', how='left')

# Cleansing
# Remove those with unitkerja_id null
df_combine = df_combine[df_combine['id_unitkerja'].notna()]


df_combine.head()

,peg_id,peg_nama,tugas_tambahan_jabatan,sekolah,id,id_unitkerja,id_unitkerja_atasan,unitkerja_nama,nama_jabatan,keselon,unkerjagrade,id_opd,unor,nip_pejabat,is_uptd,kode_simpeg
0,196709191991031005,"Drs. SAPTO WIBOWO, M.M.",KEPALA SEKOLAH SMAN 1 JONGGOL KABUPATEN BOGOR,SMAN 1 JONGGOL KABUPATEN BOGOR,10480.0,100321112000,100321110000,SMAN 1 JONGGOL KABUPATEN BOGOR,None,None,A,1003,None,None,1,None
1,196801011998021003,Drs. MARYANA,KEPALA SEKOLAH SMAN 1 CIGUDEG KABUPATEN BOGOR,SMAN 1 CIGUDEG KABUPATEN BOGOR,10280.0,100321110900,100321110000,SMAN 1 CIGUDEG KABUPATEN BOGOR,None,None,A,1003,None,None,1,None
2,196801041993021002,"ENDANG MISBAH KURNIAWAN, S.Pd.",KEPALA SEKOLAH SMAN 1 CISARUA KABUPATEN BOGOR,SMAN 1 CISARUA KABUPATEN BOGOR,10346.0,100321111300,100321110000,SMAN 1 CISARUA KABUPATEN BOGOR,None,None,A,1003,None,None,1,None
3,196807071990012001,"RD. NIA ELISA YULIANTI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 1 CIGOMBONG KABUPATEN BOGOR,SMAN 1 CIGOMBONG KABUPATEN BOGOR,10271.0,100321110800,100321110000,SMAN 1 CIGOMBONG KABUPATEN BOGOR,None,None,A,1003,None,None,1,None
4,196905191994122001,"MELWINDA FITRI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 3 CIBINONG KABUPATEN BOGOR,SMAN 3 CIBINONG KABUPATEN BOGOR,10843.0,100321113900,100321110000,SMAN 3 CIBINONG KABUPATEN BOGOR,None,None,A,1003,None,None,1,None


In [121]:
df_combine_clean = df_combine[['peg_id', 'peg_nama', 'tugas_tambahan_jabatan', 'sekolah', 'id_unitkerja']]

df_combine_clean.head()

,peg_id,peg_nama,tugas_tambahan_jabatan,sekolah,id_unitkerja
0,196709191991031005,"Drs. SAPTO WIBOWO, M.M.",KEPALA SEKOLAH SMAN 1 JONGGOL KABUPATEN BOGOR,SMAN 1 JONGGOL KABUPATEN BOGOR,100321112000
1,196801011998021003,Drs. MARYANA,KEPALA SEKOLAH SMAN 1 CIGUDEG KABUPATEN BOGOR,SMAN 1 CIGUDEG KABUPATEN BOGOR,100321110900
2,196801041993021002,"ENDANG MISBAH KURNIAWAN, S.Pd.",KEPALA SEKOLAH SMAN 1 CISARUA KABUPATEN BOGOR,SMAN 1 CISARUA KABUPATEN BOGOR,100321111300
3,196807071990012001,"RD. NIA ELISA YULIANTI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 1 CIGOMBONG KABUPATEN BOGOR,SMAN 1 CIGOMBONG KABUPATEN BOGOR,100321110800
4,196905191994122001,"MELWINDA FITRI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 3 CIBINONG KABUPATEN BOGOR,SMAN 3 CIBINONG KABUPATEN BOGOR,100321113900


In [122]:
# Get id_gps from id_unitkerja
df_combine_gps = pd.merge(df_combine_clean, db_gps, left_on='id_unitkerja', right_on='id_unitkerja', how='left')
df_combine_gps_clean = df_combine_gps[['peg_id', 'peg_nama', 'tugas_tambahan_jabatan', 'sekolah', 'id_unitkerja', 'gps_id']]
# df_combine_gps_clean['gps_id'] = df_combine_gps_clean['gps_id'].fillna(-1).astype(int).astype(str)

df_combine_gps_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   peg_id                  698 non-null    int64  
 1   peg_nama                698 non-null    object 
 2   tugas_tambahan_jabatan  698 non-null    object 
 3   sekolah                 698 non-null    object 
 4   id_unitkerja            698 non-null    object 
 5   gps_id                  691 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 32.8+ KB


In [123]:
# Remove record with gps_id null
df_combine_gps_clean = df_combine_gps_clean[df_combine_gps_clean['gps_id'].notna()]
df_combine_gps_clean['peg_id'] = df_combine_gps_clean['peg_id'].astype(str)
df_combine_gps_clean['gps_id'] = df_combine_gps_clean['gps_id'].astype(int)

df_combine_gps_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 691 entries, 0 to 697
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   peg_id                  691 non-null    object
 1   peg_nama                691 non-null    object
 2   tugas_tambahan_jabatan  691 non-null    object
 3   sekolah                 691 non-null    object
 4   id_unitkerja            691 non-null    object
 5   gps_id                  691 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 37.8+ KB


In [124]:
df_combine_gps_clean.head()

,peg_id,peg_nama,tugas_tambahan_jabatan,sekolah,id_unitkerja,gps_id
0,196709191991031005,"Drs. SAPTO WIBOWO, M.M.",KEPALA SEKOLAH SMAN 1 JONGGOL KABUPATEN BOGOR,SMAN 1 JONGGOL KABUPATEN BOGOR,100321112000,521
1,196801011998021003,Drs. MARYANA,KEPALA SEKOLAH SMAN 1 CIGUDEG KABUPATEN BOGOR,SMAN 1 CIGUDEG KABUPATEN BOGOR,100321110900,510
2,196801041993021002,"ENDANG MISBAH KURNIAWAN, S.Pd.",KEPALA SEKOLAH SMAN 1 CISARUA KABUPATEN BOGOR,SMAN 1 CISARUA KABUPATEN BOGOR,100321111300,514
3,196807071990012001,"RD. NIA ELISA YULIANTI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 1 CIGOMBONG KABUPATEN BOGOR,SMAN 1 CIGOMBONG KABUPATEN BOGOR,100321110800,509
4,196905191994122001,"MELWINDA FITRI, S.Pd., M.Pd.",KEPALA SEKOLAH SMAN 3 CIBINONG KABUPATEN BOGOR,SMAN 3 CIBINONG KABUPATEN BOGOR,100321113900,540


In [125]:
# Export to csv
df_combine_gps_clean.to_csv('df_combine_gps_clean.csv', index=False)

In [128]:
def update_gps_id_batch(df, table_name='t_jamkerja_pegawai', engine=None, dry_run=True):
    """
    Update GPS ID for multiple records in the database.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing 'peg_id' and 'gps_id' columns
    table_name : str
        Name of the table to update
    engine : sqlalchemy.engine
        Database engine (uses global engine if None)
    dry_run : bool
        If True, only shows what would be updated without actually updating
        
    Returns:
    --------
    dict
        Summary of update results
    """
    if engine is None:
        engine = globals().get('engine')
        if engine is None:
            print("❌ Error: No engine provided and no global engine found")
            return None
    
    if 'peg_id' not in df.columns or 'gps_id' not in df.columns:
        print("❌ Error: DataFrame must contain 'peg_id' and 'gps_id' columns")
        return None
    
    # Prepare update query
    update_query = f"UPDATE {table_name} SET id_gps = :gps_id WHERE nip = :peg_id"
    
    # Count updates
    success_count = 0
    error_count = 0
    errors = []
    
    if dry_run:
        print("🔍 DRY RUN MODE - No changes will be made to the database")
        print(f"Would update {len(df)} records in table '{table_name}'")
        print("\nSample updates that would be made:")
        print(df[['peg_id', 'gps_id']].head(10))
        return {
            'dry_run': True,
            'total_records': len(df),
            'sample': df[['peg_id', 'gps_id']].head(10)
        }
    
    # Execute updates in a transaction
    with engine.begin() as connection:
        try:
            for idx, row in df.iterrows():
                try:
                    result = connection.execute(
                        text(update_query),
                        {'gps_id': int(row['gps_id']), 'peg_id': str(row['peg_id'])}
                    )
                    if result.rowcount > 0:
                        success_count += 1
                    else:
                        error_count += 1
                        errors.append(f"NIP {row['peg_id']}: No matching record found")
                except Exception as e:
                    error_count += 1
                    errors.append(f"NIP {row['peg_id']}: {str(e)}")
            
            print(f"✅ Update completed!")
            print(f"   Successfully updated: {success_count} records")
            print(f"   Failed/Skipped: {error_count} records")
            
            if errors and len(errors) <= 10:
                print("\nErrors encountered:")
                for error in errors[:10]:
                    print(f"   - {error}")
                if len(errors) > 10:
                    print(f"   ... and {len(errors) - 10} more errors")
            
            return {
                'success_count': success_count,
                'error_count': error_count,
                'total_records': len(df),
                'errors': errors
            }
        except Exception as e:
            print(f"❌ Transaction failed: {e}")
            raise

# Example usage - DRY RUN FIRST (recommended)
# print("Running in DRY RUN mode first...")
# result = update_gps_id_batch(df_combine_gps_clean, dry_run=True)

# Uncomment below to actually update the database
print("\n" + "="*60)
print("ACTUAL UPDATE - This will modify the database!")
print("="*60)
result = update_gps_id_batch(df_combine_gps_clean, dry_run=False)




ACTUAL UPDATE - This will modify the database!
✅ Update completed!
   Successfully updated: 691 records
   Failed/Skipped: 0 records
